# Running Splycer

Here I demonstrate the process of running Splycer from start to finish. First, load in all of the necessary modules. In the future, `sys.path.append` will not be necessary since this will be a package built in your Python environment. Make sure you are also in the example folder, which you can change with the `os.chdir()` command listed below.

For code clarity, I turned warnings off in this notebook. Don't do this in your future code because you can get valuable warnings sometimes, especially about invalid values during comparisons.

In [1]:
import sys
import os
import pickle as pkl
import numpy as np
import pandas as pd
from xgboost import XGBClassifier

sys.path.append("R:/JoePriceResearch/record_linking/projects/deep_learning/ml-record-linking")
from record_set import RecordDataFrame
from pairs_set import PairsCOO
from feature_engineer import FeatureEngineer
from xgboost_match import XGBoostMatch
os.chdir("R:/JoePriceResearch/record_linking/projects/deep_learning/ml-record-linking/example")

import warnings
warnings.simplefilter("ignore")

## Record Set Objects

This next cell creates record set objects. A record set object is a container for record information that a linker algorithm can access with a unique identifier. In this tutorial, the record set object's internal structure is a [Pandas DataFrame](https://towardsdatascience.com/a-quick-introduction-to-the-pandas-python-library-f1b678f34673). One innovation of our code is that it does not matter what your data's internal structure is as long as you use/create a [wrapper](https://en.wikipedia.org/wiki/Wrapper_function) for it. This gives you the freedom to use the data format that is best for your linking project. I will have some documentation for how to do that, but I have created several common data structures to choose from.

(Note: the feather file format I am reading here is a special binary format that Python, R, Julia, etc. can understand. It's extremely fast, so I use it here to avoid long wait times loading the data. However, I wouldn't recommend this format for your projects since other programs can't understand it and it has some limiting design choices.)

In [2]:
delaware_1910 = RecordDataFrame(2, pd.read_feather("delaware_1910.feather", nthreads=4).set_index("index", drop=True))
delaware_1920 = RecordDataFrame(3, pd.read_feather("delaware_1920.feather", nthreads=4).set_index("index", drop=True))

Here's what the data look like. You can see there's some missing values. If I was actually running predictions, I would fix these missing values, and I should fix the missing values in these files in the future.

In [5]:
delaware_1910.df.head()

,marstat,birth_year,household,immigration,race,rel,female,bp,mbp,fbp,...,last_vec192,last_vec193,last_vec194,last_vec195,last_vec196,last_vec197,last_vec198,last_vec199,last_vec200,last_vec201
index,,,,,,,,,,,,,,,,,,,,,
8361303,1.0,1874.0,1934610.0,NaN,0,1,1,65,46,36,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8437960,0.0,1874.0,1955405.0,NaN,0,-1,1,55,12,55,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8387550,0.0,1905.0,1975755.0,NaN,0,3,1,18,18,18,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8529805,0.0,1880.0,1954502.0,1880.0,0,-1,1,10,10,10,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8529824,0.0,1878.0,1954502.0,NaN,0,-1,1,55,55,55,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Pairs Set Objects

Next, we create the pairs set. This object contains pairs of record ids that we want to predict whether they are a match, indexed by the unique identifier in the record set objects. Once again, internal data structure doesn't matter as long as you use/create a wrapper for your specific data structure. Here, I am using a [coordinate sparse matrix](https://en.wikipedia.org/wiki/Sparse_matrix#Coordinate_list_(COO)).

In [6]:
compares = pd.read_csv("delaware_compares.csv")
compares = PairsCOO(2,3, compares.index_1910.values, compares.index_1920.values, np.ones(compares.shape[0]))

## Comparing Records with Feature Engineer

Constructing the comparison engineer is the most intensive part of the pipeline. It is constructed by incrementally adding functions that measure record similarity. For each comparison, you must specify the column name(s) of the record you want to compare; the type of comparison; and any extra arguments that the comparison takes. On theme, you can use any similarity score you want as long as you create a wrapper for it.

#### Example

Suppose I wanted to compare first name using jaro-winkler score. After creating a ComparisonEngineer object `comparison_engine`, the code to do this is `comparison_engine.add_comparison("first_name", "jw")`. If I wanted to also use a commonality weight, I would use `comparison_engine.add_comparison("first_name", "jw", {"comm_weight": 'd', "comm_col": "first_comm"})`.

Below, you can see that I generate the comparison pipeline by creating lists of columns, comparisons, and extra arguments. This is the most efficient way of coding up the comparison pipeline.

In [7]:
#Construct the feature engineer
fe = FeatureEngineer()
cols = ["marstat", "race", "rel", "mbp", "fbp", "first_sdxn", "last_sdxn", "bp", "county",
        "immigration", "birth_year", ["res_lat", "res_lon"], ["bp_lat", "bp_lon"],
        [f"first_vec{i}" for i in range(2, 202)], [f"last_vec{i}" for i in range(2,202)],
        "first", "last",
        "first", "last"
       ]

#similarity functions
col_comps = ["exact match"] * 9
col_comps.extend(["abs dist"] * 2)
col_comps.extend(["euclidean dist"] * 4)
#col_comps.extend(["geo dist"] * 2)
col_comps.extend(["jw"] * 2)
col_comps.extend(["trigram"] * 2)
#col_comps = ["exact match", "exact match" ..9x.., "abs dist", "abs dist", "euclidean dist", ]
#extra arguments
col_args = list({} for i in range(5))
col_args.extend([{"comm_weight": "d", "comm_col": "first_comm"}, {"comm_weight": "d", "comm_col": "last_comm"},
                 {"comm_weight": "d", "comm_col": "bp_comm"}])
col_args.extend(list({} for i in range(7)))
col_args.extend([{"comm_weight": "d", "comm_col": "first_comm"}, {"comm_weight": "d", "comm_col": "last_comm"}] * 2)
assert len(cols) == len(col_comps) == len(col_args)

This is what the above code actually does since it's a bit confusing:

`col_comps = ["exact match", "exact match", "exact match", "exact match", "exact match", "exact match", 
              "exact match", "exact match", "exact match", "abs dist", "abs dist", "euclidean dist", "euclidean dist",
              "euclidean dist", "euclidean dist", "jw", "jw", "trigram", "trigram"]`

`col_args = [{}, {}, {}, {}, {}, {"comm_weight": "d", "comm_col": "first_comm"}, 
             {"comm_weight": "d", "comm_col": "last_comm"}, {"comm_weight": "d", "comm_col": "bp_comm"}, 
             {}, {}, {}, {}, {}, {}, {},  {"comm_weight": "d", "comm_col": "first_comm"}, 
             {"comm_weight": "d", "comm_col": "last_comm"}, {"comm_weight": "d", "comm_col": "first_comm"}, 
             {"comm_weight": "d", "comm_col": "last_comm"}]`

This way, a comparison is the built using the $i$th element of every list. For example, the first comparison added is `fe.add_comparison("marstat", "exact match", {})`. To create this, zip up all the lists and loop over them, as below.

In [8]:
for i,j,k in zip(cols, col_comps, col_args):
    fe.add_comparison(i, j, k)

You don't have to build the comparison engine like I did, but it does make the code more compact.

Here's a look at all of the available similarity functions I've coded up so far:

In [9]:
list(fe.compares_avail.keys())

['jw',
 'abs dist',
 'euclidean dist',
 'geo dist',
 'bigram',
 'trigram',
 'ngram',
 'exact match']

## Linking Model

Here I load in a model. For now, this is an old model that will give you junk predictions for our current data since I didn't bother cleaning the delaware data. Hopefully this will be improved so that you can see the actual predictions that the model is making.

In [10]:
#Construct XGBoostMatch
with open("R:/JoePriceResearch/record_linking/projects/deep_learning/ml-record-linking/model.xgboost", "rb") as file:
    model = pkl.load(file)
model.get_booster().feature_names = [f"f{i}" for i in range(19)]

This final step is to create and run a linker object. A linker object takes all of the previous parts and uses the similarity scores generated to either predict whether a record pair is a match or return a probability of a record pair being a match. Once again, the code is agnostic to which linking algorithm you use as long as the linking algorithm knows how to work with each of the objects created above.

For this notebook, I use [XGBoost](https://machinelearningmastery.com/gentle-introduction-xgboost-applied-machine-learning/), a powerful adaptation of random forests. The last step is to use `run()` and specify the name of the file that you want your predictions saved to.

In [11]:
xgb = XGBoostMatch(delaware_1910, delaware_1920, compares, fe, model)
xgb.run("test.csv")

Currently, we can run 22 million predictions per hour on a single machine, not including blocking and reading in data. 2/3 of the time is spent on the ngram comparison, so dropping that will lead to 66 million predictions per hour. You can test with your own data how much dropping ngram similarity impacts your precision and recall, but it probably won't be that big of a factor since there are many other name comparison metrics that you can use. (If you really need it, you can try optimizing my code for ngrams, which just calls a canned python module. Then submit a pull request to my github repo.)

## Conclusion

And that's the pipeline! The only step not shown is the blocking algorithm. We used SQL to block extremely quickly, but you can do it however you want. Maybe in the future we'll have a dedicated class for blocking, but that is not a priority. Use SQL if you have a very large dataset, or use Stata/Pandas for smaller datasets (merges in Stata are better than Pandas).